<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/Interests/Intro_Recommender_System_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천시스템 개요 공부

공부해보고 싶었던 분야 중 하나인 추천시스템에 대해 영상을 통해 정리하고, 추가 자료를 통해 개요 및 필요요소를 간단하게 공부하려고 한다.

영상출처> https://www.youtube.com/watch?v=6TP51jvjLsE&t=581s

참고자료> https://datascienceschool.net/03%20machine%20learning/07.01%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C.html#surprise


## 추천시스템의 구분
- content-based filtering
  
  지금까지의 사용자 행동+피드백
  사용자가 좋아하는 것과 유사한 항목 추천
- collaborative filtering

  사용자와 항목간 유사성 동시에 추천


## surprise
surprise 패키지는 추천 시스템 개발을 위한 라이브러리이다.

In [3]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 341kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618274 sha256=1f73786307f05257c535e7b467e7e653f76f8c50c4513deef60c83e9545fb3f7
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [4]:
from surprise import SVD  # 행렬의 분류에 대한 문제
from surprise import Dataset
from surprise.model_selection import cross_validate

surprise에는 다양한 데이터가 있는데 이 중, 영화 평점 데이터를 불러오려고 한다.

In [22]:
# 예제 data
data = Dataset.load_builtin('ml-100k', prompt = False)
data.raw_ratings[:10]

# User / Item / Rating / ID

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

이 데이터를 데이터 프레임으로 바꿔보면 다음과 같다.

In [7]:
import pandas as pd
df = pd.DataFrame(data.raw_ratings, columns=["user", "item", "rate", "id"])
del df["id"]
df.head(10)

,user,item,rate
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0
5,298,474,4.0
6,115,265,2.0
7,253,465,5.0
8,305,451,3.0
9,6,86,3.0


정보를 해석해보면, 196번 user는 242번 아이템에 대해 3.0의 평점을 주었다고 해석할 수 있다.

즉, 추천 시스템이란 user와 item을 바탕으로 rating을 예측하는 시스템이다.

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 **다른 사용자의 시청 목록을 비교**해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
  혹은 평점 기반으로도 추천이 가능할 것이다.

* **유사도를 기반**으로 추천

* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * **사용자의 기존 관심사항을 기반으로만** 추천 가능하다. 관심사가 바뀌는 것에 대해 반응이 늦다.

In [8]:
import numpy as np

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기.
* 이를 위해서는 adjoint matrix를 생성해야 할 것이다.
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천
* 1이 많이 겹칠수록, 높은 유사도일 것이다.


행렬을 생성하고, 선정 기준을 만든 뒤, 선택 작업을 어떻게 할 것인지 예시 코드를 살펴보려고 한다.

In [23]:
raw_data = np.array(data.raw_ratings, dtype=int)
print(raw_data)

[[      196       242         3 881250949]
 [      186       302         3 891717742]
 [       22       377         1 878887116]
 ...
 [      276      1090         1 874795795]
 [       13       225         2 882399156]
 [       12       203         3 879959583]]


In [24]:
# 인덱스를 맞추기 위한 작업  (0번부터 시작하게 만들려고 한다.)
raw_data[:,0] -= 1
raw_data[:,1] -= 1

In [25]:
n_users = np.max(raw_data[:, 0])
n_movies= np.max(raw_data[:, 1])
shape = (n_users+1, n_movies+1)  # 데이터 개수 구하기.  +1의 이유는  0번부터 시작해서.

행렬의 사이즈가 크기에 array대신에 고속 계산에 적합한 ndarry를 사용하려고 한다. ndarray를 위해서는 데이터 타입을 통일시켜야한다.

In [26]:
# 인접행렬의 생성
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = 1
adj_matrix
# 1이 있는 위치가 데이터가 있는 위치이다.

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [27]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:  # 나와 유저의 id가 다르다면
    similarity = np.dot(my_vector, user_vector)  # 이 값이 클수록 1 유사도가..
    if similarity > best_match:  # 최고 유사도가 갱신되었다면?
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 183, Best Match ID: 275


In [29]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 == 0 and log2 == 1:  # 내 벡터에서 0, 즉 나는 안보고 상대는 봤다?
    recommend_list.append(i)

print(recommend_list)

[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

혹은, 유클리드 거리를 사용해  추천가능할 것이다. 거리가 가까울수록 나랑 유사한 사용자다.

In [30]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    eucliden_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if eucliden_dist < best_match:
      best_match = eucliden_dist
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 14.832396974191326, Best Match ID: 737


In [31]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 == 0 and log2 == 1:
    recommend_list.append(i)

print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


혹은 코사인 유사도를 활용 할 수 있다.
코사인 값의 경우는 별도로 값을 생성해야 한다.
코사인 값은 클수록 유사한 방향의 벡터를 보일 것이다.
(각도가 작아지니까!)

In [32]:
def compute_cos_similarity(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)

  return dot / (norm1 * norm2)

In [33]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.5278586163659506, Best Match ID: 915


In [34]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 == 0 and log2 == 1:
    recommend_list.append(i)

print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


하지만 위의 내용은 사용자의 평가가 반영되어 있지 않다. 오로지 본 경험에 대해서만 평가가 되어있다. 사용자의 평가가 반영된다면? 평점을 반영한 인접행렬을 새로 만들려고 한다.

In [35]:
# 인접행렬의 생성
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating
adj_matrix
# 1이 있는 위치가 데이터가 있는 위치이다.

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [38]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []

for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    eucliden_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if eucliden_dist < best_match:
      best_match = eucliden_dist
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 55.06359959174482, Best Match ID: 737


In [39]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 == 0 and log2 > 4:  # 본인이 보지 않은 영화 중 가장 가까운 유저가 5점을 내린 모델을 가져오자.
    recommend_list.append(i)

print(recommend_list)

[312, 317, 384, 407, 526, 602]


## 협업 필터링(Collaborative Filtering)

* **사용자와 항목의 유사성을 동시에 고려해 추천**
* **기존에 내 관심사가 아닌 항목**이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * **학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음**
    * 추가 특성을 사용하기 어려움

In [ ]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)

이 외에에도 SVD 등을 통해서도 모델을 줄 수 있다. 즉 기존 모델을 활용하여 추천하게 된다.

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식이다.
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [40]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset


In [41]:
data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data = np.array(data.raw_ratings, dtype=int)
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [42]:
n_users = np.max(raw_data[:, 0])
n_movies = np.max(raw_data[:,1])
shape = (n_users+1, n_movies+1)
shape

(943, 1682)

In [43]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

행렬의 분해를 필요한 정보를 필터링 할 수 있을 것이다.
여기서는 SVD방식 (Singular Value Decomposition) 을 사용해보려고 한다.

In [44]:
# 사용자, "특이값", 항목
U, S, V = randomized_svd(adj_matrix, n_components = 2)
S = np.diag(S)

# S가 잠재요인이 되는 특이값 벡터가 될 것이다.

In [45]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [46]:
np.matmul(np.matmul(U, S), V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

사용자 기반 추천 방식을 사용하기 위해서는 user방식인 U를 가져와야 할 것이다.

In [47]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999942295956324, Best Match ID: 235


In [48]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 == 0 and log2 > 4:
    recommend_list.append(i)

print(recommend_list)

[281, 285, 317, 327, 418, 422, 426, 431, 482, 505, 613, 728, 734, 749]


항목기반 추천은 V를 가져와야 하는데, 행렬 곱으로 분해했으므로 Transpose를 취해야 할 것이다. SVD에 대한 자세한 내용은 추후 학습하려고 한다. (선형대수학...)

In [49]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector = -1, -1, []

for user_id, user_vector in enumerate(V.T):
  if my_id != user_id:
    cos_similarity = compute_cos_similarity(my_vector, user_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = user_id
      best_match_vector = user_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999951364145, Best Match ID: 1287


In [51]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 4:
    recommend_list.append(i)

print(recommend_list)

[0, 15, 17, 20, 22, 24, 37, 41, 42, 44, 56, 57, 76, 88, 92, 94, 95, 129, 133, 150, 156, 167, 188, 199, 208, 209, 229, 251, 252, 255, 262, 264, 275, 286, 289, 290, 293, 295, 297, 302, 304, 306, 311, 313, 323, 329, 338, 339, 342, 356, 380, 387, 389, 394, 397, 401, 415, 434, 440, 467, 471, 478, 483, 486, 502, 507, 513, 522, 525, 531, 533, 535, 544, 548, 576, 605, 613, 619, 641, 642, 647, 648, 660, 668, 675, 677, 690, 696, 704, 707, 714, 715, 720, 737, 746, 758, 766, 767, 769, 770, 803, 814, 820, 837, 863, 869, 881, 884, 886, 891, 892, 896, 900, 901, 906, 921, 923, 926, 940]


이 이외 비음수 행렬 기법도 사용 가능하다.